In [17]:
import os
import glob
import shutil
import pandas as pd
from pathlib import Path

def merge_files(folder):
    """
    Merge all files in a folder into a single DataFrame.
    """
    all_dfs = []
    folder_path = Path(folder)
    for f in folder_path.glob('*.xls'):
        try:
            df=pd.read_csv(f, header= 2, sep= '\t', encoding= 'gbk')
            all_dfs.append(df)
        except Exception as e:
            print(f"Error reading {f}: {e}")

    if all_dfs:
        return pd.concat(all_dfs, ignore_index=True)
    else:
        return pd.DataFrame()
    

def clean(text):
    text = "".join(c for c in text if not(c.isdigit()))
    for char in ['(', ')','.','@', 'PCS']:
        text = text.replace(char, '')
        text = text.replace('\\n', ' ')
        text = text.replace('W/', ' WITH ')
        text = text.replace('&', ' AND ')
        text = text.replace(' WIT ','  WITH  ')
        text = text.replace('  ',' ')
    return text.upper().strip()



In [18]:
path = input("input the folder path:").replace('\\','/')
folder = Path(path)
df_Daily_Report= merge_files(folder)

In [19]:
df_Daily_Report[['Date','POS Name','Cashier Name','Transaction No']] = df_Daily_Report[['Date','POS Name','Cashier Name','Transaction No']].ffill()
df_Daily_Report = df_Daily_Report.drop(columns= ['No data found'])
df_Daily_Report.insert(1,'DateTime',pd.to_datetime(df_Daily_Report['Date']))
df_Daily_Report['Date'] = pd.to_datetime(df_Daily_Report['Date']).dt.date
df_Daily_Report['Dishes'] = df_Daily_Report['Dishes'].apply(clean)

df_transactions = df_Daily_Report.drop_duplicates(['OR No'], keep='first')
df_transactions = df_transactions.drop(columns=['Dishes','Dish Quantities'])

df_dishes = df_Daily_Report[['OR No', 'DateTime','Date', 'POS Name', 'Cashier Name', 'Transaction No','Dishes','Dish Quantities']]

In [20]:
with pd.ExcelWriter(folder / "Daily_Report.xlsx", engine='xlsxwriter') as writer:
    df_Daily_Report.to_excel(writer, sheet_name='DailyReport', index=False)
    df_transactions.to_excel(writer, sheet_name='Transactions', index=False)
    df_dishes.to_excel(writer, sheet_name='Dishes', index=False)

In [ ]:
total_dish = df_dishes.groupby(['Date','Dishes'])['Dish Quantities'].sum().reset_index(name = 'dish_count')
total_dish[total_dish['Dishes'] == ""


,Date,Dishes,dish_count
0,2025-01-02,CHEESE STICKS,25.0
1,2025-01-02,CHICKEN BALLS,8.0
2,2025-01-02,CHICKEN MAMI,1.0
3,2025-01-02,DISPOSABLE CONTAINER,3.0
4,2025-01-02,EGG MAMI,2.0
...,...,...,...
1572,2025-04-04,SODA IN CAN,10.0
1573,2025-04-04,SODA ROYAL,1.0
1574,2025-04-04,SODA SPRITE,1.0
1575,2025-04-04,SQUID BALLS,63.0
